## 2.5.3 Challenge_ Validating Linear Regression

In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
%matplotlib inline
from scipy.stats.mstats import winsorize

In [68]:
import math
import warnings

from IPython.display import display
from sklearn import linear_model
import statsmodels.formula.api as smf

In [69]:
df = pd.read_csv("/Users/purva/Desktop/Data Science Boot Camp/table_crime data_csv.csv", header = 1)

In [70]:
df.head()

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3,Unnamed: 13
0,Adams Village,"1,861",0,0,nan,0,0,0,12,2,10,0,0.000,nan
1,Addison Town and Village,"2,577",3,0,nan,0,0,3,24,3,20,1,0.000,nan
2,Akron Village,"2,846",3,0,nan,0,0,3,16,1,15,0,0.000,nan
3,Albany,"97,956",791,8,nan,30,227,526,"4,090",705,"3,243",142,nan,nan
4,Albion Village,"6,388",23,0,nan,3,4,16,223,53,165,5,nan,nan


In [71]:
df['Property\ncrime'] = (df['Property\ncrime']).str.replace(',','')
df['Property\ncrime'] = (df['Property\ncrime']).astype(int)

In [72]:
df['Population'] = (df['Population']).str.replace(',','')
df['Population'] = (df['Population']).astype(int)
df['Robbery'] = (df['Robbery']).str.replace(',','')
df['Robbery'] = (df['Robbery']).astype(int)
df['Violent\ncrime'] = (df['Violent\ncrime']).str.replace(',','')
df['Violent\ncrime'] = (df['Violent\ncrime']).astype(int)

## Feature for population square

In [73]:
df['pop_square'] = df['Population']*df['Population']

In [74]:
df['pop_square'].sort_values()

118            276676
319            378225
136            430336
12             432964
44             580644
344            687241
287            760384
266            954529
48             960400
260            994009
25            1044484
227           1203409
87            1218816
77            1258884
34            1378276
200           1595169
256           1623076
157           1822500
293           1825201
238           1918225
23            1988100
149           2039184
242           2053489
295           2064969
128           2411809
258           2566404
208           2630884
75            2650384
186           2849344
307           2859481
            ...      
108        1867449796
334        1916425729
263        1924225956
127        2005875369
338        2008922041
134        2073436225
22         2144060416
218        2457581476
315        2489810404
301        2560663609
152        2652868036
140        3046819204
340        3313038481
314        3373518724
321       

## Feature for Robbery and Murder

In [75]:
df['feature_Robbery'] = df['Robbery']
df.loc[df['feature_Robbery'] > 0, 'feature_Robbery'] = 1

In [76]:
df['feature_Murder'] = df['Murder and\nnonnegligent\nmanslaughter']
df.loc[df['feature_Murder'] > 0, 'feature_Murder'] = 1

In [77]:
df['feature_ViolentCrime'] = df['Violent\ncrime']
df.loc[df['feature_ViolentCrime'] > 0, 'feature_ViolentCrime'] = 1

## Building Model

In [78]:
# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

In [79]:
regr = linear_model.LinearRegression()
Y = df['Property\ncrime'].values.reshape(-1,1)
X = df[['pop_square','feature_Robbery','feature_Murder']]
model1 = regr.fit(X,Y)
print('\nCoefficients: ', regr.coef_)
print('\nIntercept: ', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients:  [[1.99256120e-09 3.00052598e+02 1.18406016e+03]]

Intercept:  [40.08610471]

R-squared:
0.9859871204030131


## Validation using using holdout group

In [80]:
from sklearn.model_selection import train_test_split
# Use train_test_split to create the necessary training and test groups
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(regr.fit(X_train, y_train).score(X_test, y_test)))
print('Testing on Sample: ' + str(regr.fit(X, Y).score(X,Y)))

With 20% Holdout: -6669.758410319113
Testing on Sample: 0.9859871204030131


## Cross validation

In [81]:
from sklearn.model_selection import cross_val_score
cross_val_score(regr, X, Y, cv=10)

array([ 5.37537196e-01,  1.50364211e-01, -3.30296783e+00, -1.11013886e-01,
       -5.29245898e-01,  4.73421735e-01, -7.28201421e+03,  1.90837479e-01,
        2.43317188e-01,  5.07985153e-01])

## Revised Model

In [90]:
regr = linear_model.LinearRegression()
Y1 = df['Property\ncrime'].values.reshape(-1,1)
X1 = df[['feature_Robbery','feature_Murder']]
regr.fit(X1,Y1)
print('\nCoefficients: ', regr.coef_)
print('\nIntercept: ', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients:  [[ 339.15509404 4043.30718908]]

Intercept:  [19.60210655]

R-squared:


ValueError: shapes (348,3) and (2,1) not aligned: 3 (dim 1) != 2 (dim 0)

In [91]:
from sklearn.model_selection import train_test_split
# Use train_test_split to create the necessary training and test groups
X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(regr.fit(X_train, y_train).score(X_test, y_test)))
print('Testing on Sample: ' + str(regr.fit(X1, Y1).score(X1,Y1)))

With 20% Holdout: 0.0012904526112711423
Testing on Sample: 0.03678044719212292


In [92]:
from sklearn.model_selection import cross_val_score
cross_val_score(regr, X1, Y1, cv=10)

array([-1.77969135e+00,  4.97183806e-01, -3.70402053e+01, -6.13192375e+00,
       -2.42266415e+01, -1.03743041e+01, -1.24590780e-02, -5.83270653e-01,
       -1.49535531e+00, -6.65404400e+00])